# AUTOMATE THE PROCESS OF GRU MODEL

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, GRU, Dense, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [6]:
# Define custom R2 score metric
def r2_score_metric(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred)) 
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true))) 
    return 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())

def plot_training_history(history, target_variable):
    plt.figure(figsize=(6, 4))
    sns.lineplot(x=history.epoch, y=history.history['loss'], color='g', label='training')
    sns.lineplot(x=history.epoch, y=history.history['val_loss'], color='b', label='validation')
    plt.title('Model Loss During Training')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.show()

    plt.figure(figsize=(6, 4))
    sns.lineplot(x=history.epoch, y=history.history['r2_score_metric'], color='g', label='training')
    sns.lineplot(x=history.epoch, y=history.history['val_r2_score_metric'], color='b', label='validation')
    plt.title('R2 Score During Training')
    plt.ylabel('R2')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')
    plt.show()

def plot_predictions(y_test, y_hat_test, target_variable, mae, r2):
    # Scatter plot of actual vs. predicted values
    plt.figure(figsize=(6, 4))
    plt.scatter(y_test, y_hat_test, label='Predicted vs Actual')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, label='Perfect Prediction')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted Values for {target_variable}\nMAE: {mae:.2f}, R²: {r2:.2f}')
    plt.legend()
    plt.show()

    # Plot of actual vs. predicted over time
    plt.figure(figsize=(6, 4))
    plt.plot(range(len(y_test)), y_test, marker='o', linestyle='-', color='blue', label='Actual')
    plt.plot(range(len(y_hat_test)), y_hat_test, marker='x', linestyle='--', color='red', label='Predicted')
    plt.title(f'Actual vs Predicted Values for {target_variable}')
    plt.xlabel('Index')
    plt.ylabel(target_variable)
    plt.legend()
    plt.show()

In [9]:
import numpy as np

def train_and_evaluate_gru_model(target_variable, flag_train, _epochs=100, _batch_size=10):
    # Load your data
    mgh_data = pd.read_excel("gru-datasets.xlsx")

    # Select relevant features
    features = ['timesteps', f'{target_variable} (Predicted GL)', f'{target_variable} (Predicted NN)']
    
    X = mgh_data[features].values
    y = mgh_data[f'{target_variable} (Actual)'].values
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Reshape data for GRU input
    X_train = X_train[:, 1:].reshape((X_train.shape[0], 1, X_train.shape[1] - 1))
    X_test = X_test[:, 1:].reshape((X_test.shape[0], 1, X_test.shape[1] - 1))

    if flag_train:
        # Define the input shapes
        time_steps = X_train.shape[1]

        # Inputs
        input_q_physics = Input(shape=(time_steps, 1), name='input_q_physics')
        input_q_nn = Input(shape=(time_steps, 1), name='input_q_nn')

        # Concatenate the features
        input_features = Concatenate()([input_q_physics, input_q_nn])

        # GRU Layer
        gru_output = GRU(units=64, return_sequences=False)(input_features)

        # Dense layers to predict w_physics and w_nn
        w_physics = Dense(1, activation='sigmoid', name='w_physics')(gru_output)
        w_nn = 1 - w_physics

        # Define the model
        weights_model = Model(inputs=[input_q_physics, input_q_nn], outputs=[w_physics, w_nn])

        # Compile the model
        weights_model.compile(optimizer='adam', loss='mse', metrics=['mae', r2_score_metric])

        # Train the model
        history = weights_model.fit([X_train[:, :, 0], X_train[:, :, 1]], y_train, 
                                    epochs=_epochs, batch_size=_batch_size, validation_split=0.2, verbose=1)

        # Save the model and scaler
        weights_model.save(f'gru-models/{target_variable}_weights_model.keras')

        # Note: The scaler object was not defined in the original code; you may need to define and use it as appropriate.
        # joblib.dump(scaler, f'gru-models/{target_variable}_gru_scaler.pkl')

        # Plot training history
        plot_training_history(history, target_variable)

    else:
        # Load the model and scaler
        weights_model = load_model(f'gru-models/{target_variable}_weights_model.keras', custom_objects={'r2_score_metric': r2_score_metric})
        # scaler = joblib.load(f'gru-models/{target_variable}_gru_scaler.pkl')

        # If scaling is required, you should scale X_test and then reshape
        # For now, assuming no scaling is done or required
        # X_test = scaler.transform(X_test)
        # X_test = X_test[:, 1:].reshape((X_test.shape[0], 1, X_test.shape[1] - 1))

        # Predict weights on test data
        predicted_w_physics, predicted_w_nn = weights_model.predict([X_test[:, :, 0], X_test[:, :, 1]])

        # Calculate y_hat using the predicted weights
        y_hat_test = X_test[:, 0] * predicted_w_physics.squeeze() + X_test[:, 1] * predicted_w_nn.squeeze()

        # Evaluate the model
        mae = mean_absolute_error(y_test, y_hat_test)
        r2 = r2_score(y_test, y_hat_test)

        print(f"\nTarget Variable: {target_variable}")
        print(f"MAE: {mae:.2f}")
        print(f"R²: {r2:.2f}")

        # Plot predictions
        plot_predictions(y_test, y_hat_test, target_variable, mae, r2)


In [10]:
# Main program
train_and_evaluate_gru_model(target_variable='Temperature In', flag_train=True)

Epoch 1/100


c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


93/93 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 509.8096 - w_physics_mae: 22.5598 - val_loss: 458.9865 - val_w_physics_mae: 21.4116
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 425.0436 - w_physics_mae: 20.5861 - val_loss: 299.9285 - val_w_physics_mae: 17.3110
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 250.9687 - w_physics_mae: 15.7742 - val_loss: 118.8923 - val_w_physics_mae: 10.9008
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 89.1141 - w_physics_mae: 9.3337 - val_loss: 25.3850 - val_w_physics_mae: 5.0325
Epoch 5/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 17.1798 - w_physics_mae: 4.0449 - val_loss: 2.8032 - val_w_physics_mae: 1.6555
Epoch 6/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.8118 - w_physics_mae: 1.2667 - val_loss: 0.2082 - val_w_physics_mae: 0.3864
Epoch 7/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2726 - w_physics_mae: 0.3282 - val_loss: 0.0632 - val_w_physics_mae: 0.1562
Epoch 8/100
93/93 ━━━━━━━━━━━━━

FileNotFoundError: [Errno 2] No such file or directory: 'gru-models/Temperature In_weights_model.keras'

In [11]:
# Main program
train_and_evaluate_gru_model(target_variable='Temperature In', flag_train=False)

ValueError: File not found: filepath=gru-models/Temperature In_weights_model.keras. Please ensure the file is an accessible `.keras` zip file.